<a href="https://colab.research.google.com/github/supriyag123/PHD_Pub/blob/main/DGRNet%20Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [84]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import tensorflow as tf
import os
import math
import plotly.graph_objects as go
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout, RepeatVector, TimeDistributed, Input
from keras.models import Model
from keras import backend as K
from tensorflow.keras.optimizers import *
from tensorflow.keras.callbacks import EarlyStopping
import keras.backend as K
from keras.callbacks import Callback
import plotly
import plotly.express as px # for data visualization
import seaborn as sns

generator_multiply = 1 #each input record will generate 100 random vectors from the latent space, given the mu and sigma generated by the encoder

#from keras.utils import plot_model
#import matplotlib.pyplot as plt

#window1 = np.load(r'/content/drive/MyDrive/PHD/2021/multivariate_long_sequences_WINDOW-500.npy')
#window2 = np.load(r'/content/drive/MyDrive/PHD/2021/multivariate_long_sequences_WINDOW-1000.npy')
#window = np.concatenate((window1, window2), axis=0)
#train_data = np.load(r'/content/drive/MyDrive/PHD/2021/multivariate_long_sequences-TRAIN.npy')
#test_data = np.load(r'/content/drive/MyDrive/PHD/2021/multivariate_long_sequences-TEST.npy')



#get data
train_data = np.load(r'/content/drive/MyDrive/PHD/2024/multivariate_long_sequences-TRAIN_hourly.npy')
index = 500
#We missed i=500 from processing the iosw. So here we are dropping row with index =500
train_data= np.delete(train_data, index, axis=0)

#test_data = np.load(r'/content/drive/MyDrive/PHD/2024/multivariate_long_sequences-TEST_hourly.npy')
#all_data = np.concatenate((train_data,test_data),axis=0)
window_label = np.load(r'/content/drive/MyDrive/PHD/2024/multivariate_long_sequences_WINDOW-TRAIN_hourly.npy')
n_seq = train_data.shape[0]
window_size = train_data.shape[1]
n_features = train_data.shape[2]


plt.suptitle('Sub sequence plotting', fontsize='30')
plt.xlabel('Time', fontsize ='20')
plt.ylabel('Feature 1', fontsize='20')
plt.plot(train_data[:,:,1])
plt.show()

#---------------------------Folowing Uber paper ------------------------------------------
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(train_data, window_label, test_size = 0.2, random_state = 42)

from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error


encoder = keras.models.load_model(r'/content/drive/MyDrive/PHD/2024/VAE_SIMULATION/vae-encoder-saved-round4-latent50-dim1000.model')
decoder = keras.models.load_model(r'/content/drive/MyDrive/PHD/2024/VAE_SIMULATION/vae-decoder-saved-round4-latent50-dim1000.model')

X_train_encoded = encoder.predict(train_data)
mu, logvar, z = X_train_encoded
sigma = tf.exp(0.5 * logvar)
batch = tf.shape(mu)[0]  #number of recors / batchs
dim = tf.shape(mu)[1] #Ndimension of latent variable
store = list()
storetemp = list()

#Find average Z - a 50-dimensional vector

z_mean = np.mean(z, axis=0)

#For each batch, iterate, get the generator_multipy number of latent vectors with same window_size.
#For each z, concatenate z_mean, so it will become 100 dimensional vector

for i in range(0,batch):
  all_Z_i = tf.random.normal(shape=(generator_multiply,dim), mean = mu[i,:], stddev=sigma[i,:]) #all randorm vectors for this record i
  z_mod = np.concatenate((z_mean[None,:],all_Z_i),axis=1)
  #X_train_decoded = decoder.predict(z_mod)
  a = np.arange(generator_multiply)
  a.fill(window_label[i])
  c=np.concatenate(((z_mod,a[:,None])),axis=1)
  store.append(c)

results=np.concatenate(store,axis=0)
np.save(r'/content/drive/MyDrive/PHD/2024/labelled_concat_subsquence_data',results)
#Regression fitting
x=results[:,:-1]
y=results[:,dim]


from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

#----- get a test set from this data, to avoid further wrangling----------------
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.1, random_state = 42)


from sklearn.ensemble import IsolationForest
iso = IsolationForest(contamination=0.1)
yhat = iso.fit_predict(x_train)
# select all rows that are not outliers
mask = yhat != -1
x_train, y_train = x_train[mask, :], y_train[mask]

#Reshape now to fi LSTM
x_train = x_train.reshape((x_train.shape[0], x_train.shape[1], 1))
x_test = x_test.reshape((x_test.shape[0], x_train.shape[1], 1))

from keras.layers import Dense, Activation
from keras.models import Sequential
##!pip uninstall tensorflow
#!pip install tensorflow==2.12.0
#from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.compose import TransformedTargetRegressor
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import QuantileTransformer

model = Sequential()
model.add(LSTM(1000, input_shape=(x_train.shape[1],x_train.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(1000))
model.add(Dropout(0.2))

model.add(Dense(1000, activation = 'relu'))
model.add(Dense(units = 512, activation = 'relu'))
model.add(Dense(units = 512, activation = 'relu'))
#model.add(Dense(units = 128, activation = 'relu'))
#model.add(Dense(units = 128, activation = 'relu'))
model.add(Dense(units = 1))
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mean_squared_error'])

#model.fit(x_train, y_train,epochs=5, batch_size=50, verbose=True)

#y_pred = model.predict(x_test)

#transform

model.fit(x_train, y_train,epochs=100, batch_size=100, verbose=True)
y_pred = model.predict(x_test)
score= r2_score(y_test,y_pred)
print("r2 score is ==",score)
print("mean_sqrd_error is==",mean_squared_error(y_test,y_pred))
print("root_mean_squared error of is==",np.sqrt(mean_squared_error(y_test,y_pred)))

#plt.scatter(y_test,y_pred);
#plt.xlabel('Actual');
#plt.ylabel('Predicted');
plt.plot(y_test[0:100], color = 'red', label = 'Real data')
plt.plot(y_pred[0:100], color = 'blue', label = 'Predicted data')
plt.title('Prediction')
plt.legend()
plt.show()

from sklearn.ensemble import IsolationForest
from sklearn.metrics import mean_absolute_error


np.savetxt(r'/content/drive/MyDrive/PHD/2024/MLPOutput/pred_2.csv',y_pred_t)
np.savetxt(r'/content/drive/MyDrive/PHD/2024/MLPOutput/real_2.csv',y_test)
print("MAE is==",mean_absolute_error(y_test,y_pred))

#---------------------------------Without VAE------------------------------------

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(train_data, window_label, test_size = 0.2, random_state = 42)

from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error


x_train = x_train.reshape((x_train.shape[0], x_train.shape[1]*n_features))
x_test = x_test.reshape((x_test.shape[0], x_test.shape[1]*n_features))

from sklearn.covariance import EllipticEnvelope
ee = EllipticEnvelope(contamination=0.05)
yhat = ee.fit_predict(x_train)
# select all rows that are not outliers
mask = yhat != -1
x_train, y_train = x_train[mask, :], y_train[mask]


x_train = x_train.reshape((x_train.shape[0], window_size, n_features))
x_test = x_test.reshape((x_test.shape[0], window_size, n_features))

from keras.layers import Dense, Activation
from keras.models import Sequential
##!pip uninstall tensorflow
#!pip install tensorflow==2.12.0
#from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.compose import TransformedTargetRegressor
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import QuantileTransformer

model = Sequential()
model.add(LSTM(1000, input_shape=(x_train.shape[1],x_train.shape[2])))
model.add(Dense(1000, activation = 'relu'))
model.add(Dense(units = 512, activation = 'relu'))
model.add(Dense(units = 512, activation = 'relu'))
#model.add(Dense(units = 128, activation = 'relu'))
#model.add(Dense(units = 128, activation = 'relu'))
model.add(Dense(units = 1))
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mean_squared_error'])

#model.fit(x_train, y_train,epochs=5, batch_size=50, verbose=True)

#y_pred = model.predict(x_test)

#transform

model.fit(x_train, y_train,epochs=100, batch_size=50, verbose=True)
y_pred = model.predict(x_test)
score= r2_score(y_test,y_pred)
print("r2 score is ==",score)
print("mean_sqrd_error is==",mean_squared_error(y_test,y_pred))
print("root_mean_squared error of is==",np.sqrt(mean_squared_error(y_test,y_pred)))

#plt.scatter(y_test,y_pred);
#plt.xlabel('Actual');
#plt.ylabel('Predicted');
plt.plot(y_test[0:100], color = 'red', label = 'Real data')
plt.plot(y_pred[0:100], color = 'blue', label = 'Predicted data')
plt.title('Prediction')
plt.legend()
plt.show()

from sklearn.ensemble import IsolationForest
from sklearn.metrics import mean_absolute_error


np.savetxt(r'/content/drive/MyDrive/PHD/2024/MLPOutput/pred_2.csv',y_pred_t)
np.savetxt(r'/content/drive/MyDrive/PHD/2024/MLPOutput/real_2.csv',y_test)
print("MAE is==",mean_absolute_error(y_test,y_pred))

#########Isolation forest-------------------------#
iso = IsolationForest(contamination=0.1)
x_train, x_test, y_train, y_test = train_test_split(train_data, window_label, test_size = 0.2, random_state = 42)

x_train = x_train.reshape((x_train.shape[0], x_train.shape[1]*n_features))
x_test = x_test.reshape((x_test.shape[0], x_test.shape[1]*n_features))
yhat = iso.fit_predict(x_train)
mask = yhat != -1
x_train, y_train = x_train[mask, :], y_train[mask]
x_train = x_train.reshape((x_train.shape[0], window_size, n_features))
x_test = x_test.reshape((x_test.shape[0], window_size, n_features))

model = Sequential()
model.add(LSTM(1000, input_shape=(x_train.shape[1],x_train.shape[2])))
model.add(Dense(1000, activation = 'relu'))
model.add(Dense(units = 512, activation = 'relu'))
model.add(Dense(units = 512, activation = 'relu'))
#model.add(Dense(units = 128, activation = 'relu'))
#model.add(Dense(units = 128, activation = 'relu'))
model.add(Dense(units = 1))
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mean_squared_error'])

#model.fit(x_train, y_train,epochs=5, batch_size=50, verbose=True)

#y_pred = model.predict(x_test)

#transform

model.fit(x_train, y_train,epochs=100, batch_size=50, verbose=True)

#remove outlier from test data to get a better prediction

x_test = x_test.reshape((x_test.shape[0], x_test.shape[1]*n_features))
yhat = iso.fit_predict(x_test)
mask = yhat != -1
x_test, y_test = x_test[mask, :], y_test[mask]
x_test = x_test.reshape((x_test.shape[0], window_size, n_features))
y_pred = model.predict(x_test)
score= r2_score(y_test,y_pred)
print("r2 score is ==",score)
print("mean_sqrd_error is==",mean_squared_error(y_test,y_pred))
print("root_mean_squared error of is==",np.sqrt(mean_squared_error(y_test,y_pred)))


np.savetxt(r'/content/drive/MyDrive/PHD/2024/MLPOutput/pred_wo-vae2.csv',y_pred)
np.savetxt(r'/content/drive/MyDrive/PHD/2024/MLPOutput/real_wo-vae2.csv',y_test)

#########----------------------------------  Now run the VAE to general more sample -------------------------------------------------------------------------#


from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(train_data, window_label, test_size = 0.2, random_state = 42)

encoder = keras.models.load_model(r'/content/drive/MyDrive/PHD/2024/VAE_SIMULATION/vae-encoder-saved-round5-latent50-dim1000.model')
decoder = keras.models.load_model(r'/content/drive/MyDrive/PHD/2024/VAE_SIMULATION/vae-decoder-saved-round5-latent50-dim1000.model')

#Encode x_train
X_train_encoded = encoder.predict(x_train)
#Decode X_train_encoded

#Calculate reconstruction error
mu, logvar, z = X_train_encoded
sigma = tf.exp(0.5 * logvar)
X_train_decoded = decoder.predict(z)
reconstruction_loss = tf.reduce_mean(tf.keras.losses.binary_crossentropy(x_train, X_train_decoded))

batch = tf.shape(mu)[0]  #number of recors / batchs
dim = tf.shape(mu)[1] #Ndimension of latent variable
#store = list()
#storetemp = list()

#Merge x_train, and also output of decoder (sample generated with VAE)
x_train_total = np.vstack((x_train, X_train_decoded))

#Now generate window label for VAE data - basically same as original window size array, vstacked
y_train_total = np.vstack((y_train, y_train)).flatten().reshape(x_train_total.shape[0],1)

x_train = x_train_total.reshape((x_train_total.shape[0], x_train_total.shape[1]*n_features))
y_train = y_train_total

x_test = x_test.reshape((x_test.shape[0], x_test.shape[1]*n_features))

from sklearn.covariance import EllipticEnvelope
from sklearn.ensemble import IsolationForest

iso = IsolationForest(contamination=0.1)
yhat = iso.fit_predict(x_train)
#ee = EllipticEnvelope(contamination=0.1)
#yhat = ee.fit_predict(x_train)
# select all rows that are not outliers
mask = yhat != -1
x_train, y_train = x_train[mask, :], y_train[mask]


x_train = x_train.reshape((x_train.shape[0], window_size, n_features))
x_test = x_test.reshape((x_test.shape[0], window_size, n_features))

from keras.layers import Dense, Activation
from keras.models import Sequential
##!pip uninstall tensorflow
#!pip install tensorflow==2.12.0
#from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.compose import TransformedTargetRegressor
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import QuantileTransformer


from sklearn.ensemble import IsolationForest
from sklearn.metrics import mean_absolute_error


np.savetxt(r'/content/drive/MyDrive/PHD/2024/MLPOutput/pred_2.csv',y_pred_t)
np.savetxt(r'/content/drive/MyDrive/PHD/2024/MLPOutput/real_2.csv',y_test)
print("MAE is==",mean_absolute_error(y_test,y_pred))


#----------------Random forest ----------------------------------
###### Random forrest ergression
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.compose import TransformedTargetRegressor
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import PowerTransformer
from sklearn.linear_model import HuberRegressor
 # create regressor object
#regressor = RandomForestRegressor(n_estimators = 50, random_state = 0)
# prepare the model with input scaling
pipeline = Pipeline(steps=[('power', PowerTransformer()), ('model',RandomForestRegressor(n_estimators = 100, random_state = 0))])
#pipeline = Pipeline(steps=[('power', PowerTransformer()), ('model',HuberRegressor())])

# prepare the model with target scaling
model = TransformedTargetRegressor(regressor=pipeline, transformer=PowerTransformer())

# fit the regressor with x and y data
pipeline.fit(x_train,y_train)
y_pred = pipeline.predict(x_test)  # test the output by changing values
# creating an object of LinearRegression class
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
# predicting the accuracy score

score=r2_score(y_test,y_pred)
print("r2 score is ==",score)
print("mean_sqrd_error is==",mean_squared_error(y_test,y_pred))
print("root_mean_squared error of is==",np.sqrt(mean_squared_error(y_test,y_pred)))
plt.scatter(y_test,y_pred);
plt.xlabel('Actual');
plt.ylabel('Predicted');


plt.plot(y_test[5000:5049], color = 'red', label = 'Real data')
plt.plot(y_pred_t[5000:5049], color = 'blue', label = 'Predicted data')
plt.title('Prediction')
plt.legend()
plt.show()




Epoch 1/100
149/252 [================>.............] - ETA: 29s - loss: 0.9170 - mean_squared_error: 0.9170

KeyboardInterrupt: 